# Filecoin Chain Data via JSON-RPC

Date: _2021-06-14_

[Filecoin has support for JSON-RPC API](https://filecoin-shipyard.github.io/js-lotus-client/intro/json-rpc.html), a standardized way to encode remote procedure calls in JSON. Projects such as Bitcoin and Ethereum have adopted it for use in their APIs.

For example, when you run the Lotus node with lotus daemon, it will listen for connections on TCP port 1234 (by default).

Applications may connect to this port and issue JSON-RPC calls. For example, using CURL, if we want to call the Filecoin.Version method, we might do this:

```bash
curl -X POST \
  -H "Content-Type: application/json" \
  --data '{ 
      "jsonrpc": "2.0", 
      "method": "Filecoin.Version", 
      "params": [], 
      "id": 1 
    }' \
  http://127.0.0.1:1234/rpc/v0 
```

Since Filecoin doesn't have a Python client and tooling around CAR files is still in development, we'll use [the JSON-RPC API to get data from the Filecoin chain](https://lotus.filecoin.io/reference/basics/api-access/).

In [1]:
import requests
import pandas as pd


In [2]:
ENDPOINT_URL = "https://api.node.glif.io"  # Another alternative is "https://lotus.miner.report/mainnet_api/0/node/rpc/v0"


In [3]:
def filecoin_rpc(method: str, params):
    """
    Call a Filecoin JSON-RPC method with the given parameters.
    """
    payload = {
        "jsonrpc": "2.0",
        "method": method,
        "params": params,
        "id": 1,
    }
    response = requests.post(ENDPOINT_URL, json=payload).json()
    if "error" in response:
        raise Exception(response["error"])
    return response["result"]


def get_tipset_by_height(height: int):
    """
    Get a tipset by its height.
    """
    return filecoin_rpc("Filecoin.ChainGetTipSetByHeight", [height, None])


With this we're ready to go. First, let's see the head of the chain:

In [4]:
chain_head = filecoin_rpc("Filecoin.ChainHead", [])
latest_epoch = chain_head["Height"]
print(f"Latest epoch is {latest_epoch}")


Latest epoch is 2967765


In [64]:
pd.json_normalize(chain_head["Blocks"])


,Miner,BeaconEntries,WinPoStProof,Parents,ParentWeight,Height,Timestamp,ForkSignaling,ParentBaseFee,Ticket.VRFProof,ElectionProof.WinCount,ElectionProof.VRFProof,ParentStateRoot./,ParentMessageReceipts./,Messages./,BLSAggregate.Type,BLSAggregate.Data,BlockSig.Type,BlockSig.Data
0,f01777777,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 4, 'ProofBytes': 'jbAbjqUp34ot/...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,qBguMRwOCZIr6R3i1wyPYRrL7UkSZRbcIy5Vo5tyGPnuck...,1,pP/X/5xcg3Ej05zUwgmhwU6Pi1n0CjL/ndCaMJMZLYU9OI...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzacedioyzrsqlnlixbczstsskzjlj6c7gfwm4vp7...,2,kNrhxvl6hKsQpIbN5Xx/IsQrXEUhcNzItoMPLssGoimlFz...,2,qkOwjqkp/oM1b5+VXulPUyZhGoJEYw4RDwoACK7bBsYsn1...
1,f01945216,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 3, 'ProofBytes': 'mWqQoLMu/HR5a...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,kfDQkOqlPdgDHgCmsBOt6U05SfahiBpIXOgljP8Agsu1lW...,1,tQuiJxY2J56ml1OPF9vzwTQ8VDLoxSCzdWI/sQB5cXncPQ...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzacebqwmvftufwcoe5yk5joki7prrz4bohynvfqm...,2,p/t0uBEzIXvCpzm6n5ZTeYctnDdzfZnhQ7q5OIby8nU4wm...,2,iwzbfDUTyGlOElQLC1JKst8HPDOP1+4FhKAit5qf1XMxun...
2,f01889512,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 4, 'ProofBytes': 'p6GEmFbVe2cc0...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,hTfkJgtGdl5/WEPiN+NeQBWZyDWPwOdIV/puBZ19eqyb9p...,1,uPzJP1x+W19njRi49YuN9RCDqCktEEMu8KKviBzn86lcq1...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzacec5ok37dwydjkf76uup3fau6e3rh2saezttbn...,2,k48kicfKWQ2cLgemP7/PXvUPdv+ApV6+SZ7OoAvWxJA7kh...,2,tVK+MmLL+Zc11mcDGqsVnpI0j+37LEqbdd4Ilvc3Hd+51H...
3,f02127257,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 4, 'ProofBytes': 'maMCZ4uziHmV5...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,s2AtNjbDMAj+WBfzKUKSWZxliZrje7tX4X3XwAeDQiR0fM...,1,pkNcrVzsGr5vzxhNLPDk6uBZLrwy9LSodhDBKz1G22VNDv...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzaceatlma645ji5o4233jxiwqceldx4r6dg4i5kl...,2,sbb4Nssgjd28HrcsehfgyutXqjvdr3tKZL6+OJU2DDbw45...,2,g0Y7fTB37Usm0UiTHkYfVv2B39IUR7UlD/EzEJcB8hMXX6...
4,f0867300,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 3, 'ProofBytes': 'qyyuVYNR/uinA...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,g3LmIDR0xEZHVYzTzR44/wTFaYnCd+Dxwm5YZZIWjYJS4u...,1,gg9gqWV/wj85Iitf6sO7mO+bv/IeiT5VP7/24z0aTxXhLe...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzacecmd2tcjgx6xxw2drqken7is56cmmunbkzozy...,2,l/STkmYziTtq1rGngLw/RyWOJAE/cI65e7awaX+xRUrQQ8...,2,g42mGqH9ugv7SDAKqX1tRnrDu9CS66WXu71l8LotwB01kU...
5,f01975338,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 3, 'ProofBytes': 'ipgtQdmncW7Wj...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,kA1Xa3of+el1ehibDHT9V4K2t2XPatgENKmLgRLlm0sAi2...,1,rS6dxhS3ku00VuwCf2JuBT3JIE4Yvt4V+Xg1TYAhRNGZLT...,bafy2bzaceco5bzk37qir4fc62lnjkiir47lj64kc5rlij...,bafy2bzacec7w5xagjrbeugqxscmvhljvil6selmvtlu6z...,bafy2bzacebcugxmtmlcx7sxy22mopilykjlxvfsntqvgb...,2,i9ZhtcFB0yFu9BbI1FrwZigyav3k2FKA+plZCEWvmyGNDR...,2,mMRMoXRT3pBlwa/CfSSuEEx/4o7WkJ5vzh2KLh6AWTWz7F...
6,f0136399,"[{'Round': 3044027, 'Data': 'jsqnJfP8uSHqWnRPD...","[{'PoStProof': 4, 'ProofBytes': 'lQvtJG0vsKvWG...",[{'/': 'bafy2bzaceacxbckrwx45ntjp7nwnfxx5bzwie...,69520993633,2948182,1686751860,0,1034486241,qMr9HIgMgfHNh4Tz7S2OXLxduP+19GP7PwMkp0b+xQkglx...,1,jonTVrtABXm4Y7lb3O/uJDqxExSQIaLJzh7Q3T17tQLCQI...,bafy2bzaceco5bz

We can get all active miners.

In [65]:
state_miners = filecoin_rpc("Filecoin.StateListMiners", [chain_head["Cids"]])


In [66]:
print(f"Total number of miners is {len(state_miners)}")


Total number of miners is 599304


We could, in theory, get the power of each miner by calling Filecoin.StateMinerPower, but it is be very slow. 

In [68]:
# from tqdm.auto import tqdm

# mp = []
# for miner in tqdm(state_miners):
#     m = filecoin_rpc("Filecoin.StateMinerPower", [miner, chain_head["Cids"]])
#     m["MinerID"] = miner
#     mp.append(m)

# pd.json_normalize(mp).sample(5)


We can get messages from random blocks too!

In [69]:
block = chain_head["Cids"][0]

block_messages = filecoin_rpc("Filecoin.ChainGetBlockMessages", [block])
pd.json_normalize(block_messages["SecpkMessages"])


,Message.Version,Message.To,Message.From,Message.Nonce,Message.Value,Message.GasLimit,Message.GasFeeCap,Message.GasPremium,Message.Method,Message.Params,Message.CID./,Signature.Type,Signature.Data,CID./
0,0,f13czx5l52ujqpluqur5lbhyoyqn2feqqir6pkqki,f1zk5enkukxuyl4vdy7zxruhktmjepxjb7m6vv5pa,23758,54419827200000000000,15000000,5167034692,258061572,0,None,bafy2bzacedoixxab6qy4ef5igrh6uylrqlfmorji523ib...,1,n3YC+Csj4fvDX/EhouL5dQYPqNBxCkJ8WY52kIb13aZPOJ...,bafy2bzacebm6cv7cdjwhngrekqp3u2ww2i4wgrmyblghr...
1,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f15d3nhmg4gq5rbretn3tsro5yefvysvpprbw6ama,37,2983489820000000000,3059156,46106420400,78302002,0,None,bafy2bzacea62uffekc55h7aphvciisdsji4unjz2uzrzf...,1,GkBwZIXB8x/lB2Tg7QHMdIBrI+7CRQbsIguck+pX+RVTJq...,bafy2bzacec6bk4l6nbh4ilrlm5p4nw6x5adaiserbvvpq...
2,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1ulaw56ozhnpn72qdo2hpz6ielvgofycn3spowhi,10,11998815050000000000,3055906,46106215240,78240454,0,None,bafy2bzacecx34unm2s433gfomogweuk7zaaowixgr5654...,1,DkwOb8e1K6GvK/H0TvLW42065LqMcuFOOLzk68Q5PJ1paW...,bafy2bzaceahyvuhiistskztj7xco7htzgaet2ykvhu4kt...
3,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1sz7ec3t6ranyeyqou3a6vmlvbqia6rsmwdtnnzy,13,2997208650000000000,3055906,46103796556,77514849,0,None,bafy2bzacedn2vq7mgsqvraypbftl56fhpftk6uwhrvbvw...,1,EINbCRUD4TI96TVrUUHtLYfgxOawKWDenlvdxcA9H1oLqQ...,bafy2bzacece2o4qnbe4xfvrjogefaven6cpyrx23srajs...
4,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1rwl2qs3jlhlgy6b7xrlinmkqqyz6jf2bsseigaa,40,7959151700000000000,3059156,46103776832,77508932,0,None,bafy2bzaceajrhz2vrjh76xeichxazl56fgw4574nlvvhw...,1,wxbqJDUNMOBbjc/PB4f1HWWFivUlMeSCn/r5nFOJINtzAh...,bafy2bzacedzc2vcmmnlzwql7lk32gjh3skvv636ufxarh...
5,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1zvkok7jdyyk3f6huynq5oxc2w55oxkybn4vquba,1,7093530130000000000,3055906,46103753728,77502001,0,None,bafy2bzacec4xekrqunvyueplmygcq7o66a534aqjtgiw6...,1,7KGZOlYGpiuDX1PC6mRndVrVcAcS11p9jB3LdTLvAAF1mq...,bafy2bzacec4gpun7iomeliyuwjifhvsonq5yplwpvhft7...
6,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1i7dwuq3cqk3xa4a4w2vjg5l52notai7yaswx5wa,1,798826780000000000,3055906,46103597016,77454987,0,None,bafy2bzacecttjqn6b5acjbkf22imzx42n475ory2ebesu...,1,wA4Ijn4WATV1eg5+S1ieEdvEQbutVsBxOxE9lG3tu/Base...,bafy2bzacebojmtoycz3aukrwhypk745kba2fvq6fwl2qo...
7,0,f1ys5qqiciehcml3sp764ymbbytfn3qoar5fo3iwy,f1mu6hgqedrjybmntxwb2ghtge46vxs7tfkjwl6ia,19,4376760970000000000,3055906,46102967620,77266168,0,None,bafy2bzacebtfrrz3kpzfujguqj233klu74wmabwsahzor...,1,ZLmlK2X7fznzGkA0g2YB8u7fRH60t4KMgDFh1Mj3OBQiqB...,bafy2bzaceapcvw37j2bjivfouogrqqgqys2zifkjglxey...
8,0,f410fk27kkiq27z4wqidktmqolw6hqwcy4nhhhbvh5bq,f410fqdg6ccovwwf357zlpkxrtzcv2bve3y6x6ajcxbq,49,500000000000000000,2280436,1811043585,75536620,3844450837,None,bafy2bzacebsmvlttclog5ufmmrrdhrnx4ewnnqer5x4xf...,3,qtoTsLNAEdKU1Xp6RJ4M1ZBKjU0iexS6UfW/eo7sZ3V1UD...,bafy2bzacebkyzrzc4qrojj2pizz46e7st3mzunjirtqmy...
9,0,f05,f1wkntaokupidewobwp5wv3fqxhqmdwpma4c65hii,10753,0,1627739163,1228698089,64946244,4,gYiCi9gqWCgAAYHiA5IgIDSJckakN14VgjGtKQmBYBkzCt...,bafy2bzacedfeobxgq65tfy57cigjxukvn6m4noxia2i52...,1,g+AEWakCPfn9ssk9QkhQpEDfuriYjIiOdIh7K/cyVE8zZD...,bafy2bzacebjbae6knpzc562shi2hflzyrr7t7krcrcet5...


Messages from 5 heights ago:

In [74]:
pd.json_normalize(get_tipset_by_height(chain_head["Height"] - 5)["Blocks"])


,Miner,BeaconEntries,WinPoStProof,Parents,ParentWeight,Height,Timestamp,ForkSignaling,ParentBaseFee,Ticket.VRFProof,ElectionProof.WinCount,ElectionProof.VRFProof,ParentStateRoot./,ParentMessageReceipts./,Messages./,BLSAggregate.Type,BLSAggregate.Data,BlockSig.Type,BlockSig.Data
0,f01839495,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'rAtmt0uFj/l/d...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,pEPiM6+Lor11hW/nTl4rgju/Ux6027+vbMqaOl8KjKlGkz...,1,qgyPhrQpwhVgDHhAPOSib4Zyg/WcwKoLLPdCh8SE/NLto/...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzacedzqhrmlvhrbffzx73gqbmm6ws23z7im3coak...,2,iwbEM6TOF4sogEdzEr+NM+OUMTAZ/Hmg2eXA0cM4jmDlG0...,2,lT33uE2+YFE1UFHGgITlkeS5WbdWLI6sSCdqBXRix88HMV...
1,f01920091,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 4, 'ProofBytes': 'k+Z7o2QOObIuK...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,tQVWDRLtXYgL/j+z9PWIUGhFkjZv16meZ5ayAubfV0ejsv...,1,tfSmuwgIAVljWd9nPBw5xNZzmNAE2LfFZYvJur5TZjmtKY...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzaceck2vf5kdkwubstgbgn23tzmmzshwmxwuvqtl...,2,sNMqciVZmvtjjpIC7tgCeIP+rfBv88WzK/ejJlX7AEG3Is...,2,qpruh9alqG0YvX9NXL59mOh51eMyhiqSfvtXTyn0awOqPn...
2,f01228108,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'klxxVKPJ2aCPX...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,hb2ZiPPAYaUFTyvUZJTl3O+G3udy25tZ1z8Sdtq60ghCgn...,1,jO2wAplJbsM3apX24f07Jj9O5dQVxwXktXLmQivr5zoBeN...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzacecptz57vc4oty23yxrnyunrycnsjqjyavtyq6...,2,sKE5zcRffuQ4w5O9AKUrXqY/qEP9GTV+Ql8SRh4Dot8uGG...,2,i+bUuG3Vh7Nw26r0dnNqVz+MUnxVyd0Z0igS794TwYjJo0...
3,f01794032,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'rd0n0TRv4w7CK...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,ls4krGkfmhQlSxv8H7sVHsrJm1z+ncticA1NoY0HRmTZbL...,1,kryt0aVRV5Fn3GXLDpQnb5B8yd8JWjSy4hq0Qix2vZBmPM...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzacecpqx62p7ythojrfidifmdjq3cugkzdpscum6...,2,q+uWAzx3kuTORhV/7Lcxksrak5JNcCE5UlYnS0hiZywFeN...,2,lw4q7rVJj4IbPoccDLUq66Rk10RHjCKEquN7N76SrPMWNG...
4,f01815391,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'sb5bWh1Ztb+J4...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,lih0LN/RwvhY4Ssb5dc6k+a9bGzB7AYzUa8z3c3ljfflA/...,1,lSacKRUNBtFmjzIXQaGs46AHWKQZdWi8Sd89IAMQtabvVn...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzacebczfq2jtulhzb5n3t4e5kvvot2kykau7x4ms...,2,rPLwGxeNL31d+KrrOfRn0+72OEkyAraNwbq5Lm5czUOig8...,2,s1a4XBiH9qH9yFGNQAOSQ2Py/a6dTEPwIb7uxs+6sLld/g...
5,f02035238,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'qgKx+CRPN2iP6...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,jBGH5LIO+jmPCiNLZS7xyDkSOIvZwNECgBQ5U+T3aImthw...,1,qsCFotvYIdAj08RVZk+IRDIfd7cqtEN3Rxqqbo1kueDw7H...,bafy2bzacedru246p3cnapplkhrhs6ffjoqth3rl5h537g...,bafy2bzacecizodixvl2eb6jhji5rrngenvvrr4rqv6ke2...,bafy2bzacede2rxv2oirev4yolkhodjbzjtsmkg6o7afef...,2,uHjm6+X3Zw/QBt1vbvv3T7Ra+UfftUVRdcOBGGLVs6r4Ko...,2,oDYq6cxiG3Umt/CnegugVsF0S6UL0DUtBcW/v2UO3ojZXE...
6,f0427989,"[{'Round': 3044022, 'Data': 'q0a1FuP1bGMe1mDv6...","[{'PoStProof': 3, 'ProofBytes': 'tTnQbvLql+TOU...",[{'/': 'bafy2bzacebowjoilxsi5vjmjmc46wwaksl75u...,69520874032,2948177,1686751710,0,1174502781,hb/dAd9S1WggmkgQLxo9ZMNV1pzC+D8Jz+KSYW5u03h3HZ...,1,paK7nzhDIG6TmMui1DN/bau5n8mkhKN0xeQK/O+A0mZNqe...,bafy2bzacedru2